# FP Growth Approach to determine Frequent Item Sets

In [ ]:
#open file to write the intermediate updates
output_file = open("Z:\Big Data and Data Mining\Assignments\CourseWork1\FPGrowth.txt",'w')

In [ ]:
#Import libararies
from timeit import default_timer as timer
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
#Read minimum support
freq_itemset = []
min_support = input("Enter the value of minimum support\n")

In [ ]:
#Class FP Tree Node
class FPTreeNode:
    def __init__(self, name,count,parent_node):
        self.name = name
        self.count = count
        self.link = None
        self.parent = parent_node
        self.children = {}
        
    def __str__(self, level=0):
        ret = "\t"*level+repr(self.name)+"\n"
        for child in self.children:
            ret += (self.children[child]).__str__(level+1)
        return ret
    
    def __repr__(self):
        return '<fpnode>'
    
    def increment_counter(self, count):
        self.count += count

In [ ]:
#Add fp node to a specific position
def add_fp_nodes(item, fptree, header, count):
    if item[0] in fptree.children:
        fptree.children[item[0]].increment_counter(count)
    else:
        fptree.children[item[0]] = FPTreeNode(item[0], count, fptree)
        if header[item[0]][1] == None:
            header[item[0]][1] = fptree.children[item[0]]
        else:
            while (header[item[0]][1].link != None):
                header[item[0]][1] = header[item[0]][1].link     
            header[item[0]][1].link = fptree.children[item[0]]
    if len(item) > 1:
        add_fp_nodes(item[1::], fptree.children[item[0]], header, count)

In [ ]:
#Traversal
def traverse(leaf, path):
    if leaf.parent != None:
        path.append(leaf.name)
        traverse(leaf.parent,path)

In [ ]:
#Function to read a file, and build fptree based on the data provided
def read_and_createTree(set_dictionary,read_from_file):
    if(read_from_file == 1):
        file_content = list()
        with open('Z:\Big Data and Data Mining\Assignments\CourseWork1\GroceryStore.csv') as f:
            rows = f.readlines()
        for i in rows:
            line = i.strip().strip(",")
            file_content.append(line.split(','))

        set_dictionary = {}
        for j in file_content:
            if frozenset(j) in set_dictionary.keys():
                   set_dictionary[frozenset(j)] += 1
            else:
                   set_dictionary[frozenset(j)] = 1

    header = {}
    for k in set_dictionary:
        for key in k:
            header[key] = header.get(key,0) + set_dictionary[k]
    
    for l in list(header):
        if(header[l] < int(min_support)):
            del(header[l])
    
    freq_items = set(header.keys())
    
    for m in header:
        header[m] = [header[m], None]
    
    Tree = FPTreeNode('Null',1,None)
    for item_set_fptree,count in set_dictionary.items():
        tx = {}
        for item in item_set_fptree:
            if item in freq_items:
                tx[item] = header[item][0]
        if len(tx) > 0:
            ordered_itemset = [v[0] for v in sorted(tx.items(), key=lambda p: p[1], reverse=True)]
           #the nodes are updated into tree
            add_fp_nodes(ordered_itemset, Tree, header, count)  
    return Tree, header

In [ ]:
#Function to mine the obtained fptree adn generate frequent item sets
def mining(fptree, header, preset, freq_itemset):
        FP = [x[0] for x in sorted(header.items(),key=lambda p: p[1][0])]
        for base in FP:
            frequent_item_set = preset.copy()
            frequent_item_set.add(base)
            freq_itemset.append(frequent_item_set)
            pattern_base = {}
            while header[base][1] != None:
                path = []
                traverse(header[base][1],path)
                if len(path) > 1:
                    pattern_base[frozenset(path[1:])] = header[base][1].count
                header[base][1] = header[base][1].link
            conditional_Tree, conditional_header = read_and_createTree(pattern_base,0)
            
            if(conditional_header != None):
                mining(conditional_Tree, conditional_header,frequent_item_set, freq_itemset)

In [ ]:
start = timer()
fp_Dict = {}
fptree,header = read_and_createTree(fp_Dict,1)
preset = set([])
mining(fptree,header,preset,freq_itemset)
end = timer()

In [ ]:
fptree_str =str(fptree)
freq_item_str = str(freq_itemset)
total_time = end-start

output_file.write("1.minimum support :" + min_support)
print("1.minimum support:"+min_support)
output_file.write("\n\n2.FP Growth Approach:")  
print("\n\n2 FP Growth Approach:")
output_file.write("\n\n\n3.FP_tree:\n"+fptree_str)
print("\n\n\n3.FP_tree:\n"+fptree_str)
output_file.write("\n\n\n4.Frequent_sets:\n"+str(freq_itemset))
print("\n\n\n4.Frequent_sets:\n"+str(freq_itemset))

output_file.write("\n\n5.Time taken for execution using bruteforce approach:"+str(total_time)) 
print("\n\n5.Time taken for execution using FP Growth approach:" + str(total_time))

In [ ]:
#Close output file after writing
output_file.close()